In [1]:
import gc, argparse, sys, os, errno
%pylab inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set_style('whitegrid')
import h5py
import os
from tqdm import tqdm_notebook as tqdm
import scipy
import sklearn
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


/home/xupeng/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
cd ..

/home/xupeng/projects/signal


In [32]:
ls citydata/

10feature_counts100000test_counts1000point_counts/       non_rate_10.txt
10feature_counts1000test_counts1000point_counts/         non_rate_20.txt
20feature_counts100000test_counts1000point_counts/       non_rate_30.txt
30feature_counts100000test_counts1000point_counts/       non_rate_40.txt
distance.txt                                             non_rate_50.txt
__DOA__10feature_counts1000test_counts1000point_counts/  relation_10.txt
features_10.txt                                          relation_20.txt
features_20.txt                                          relation_30.txt
features_30.txt                                          relation_40.txt
features_40.txt                                          relation_50.txt
features_50.txt


In [7]:
distancedata = pd.read_table('citydata/distance.txt',sep=' ',header=None)

In [10]:
distancedata.head()

,0,1,2,3,4,5,6,7,8,9,...,1490,1491,1492,1493,1494,1495,1496,1497,1498,1499
0,0.000000,22.360680,36.055513,28.284271,41.231056,58.318093,51.000000,64.039051,106.306162,86.029065,...,1887.884795,1910.028534,1886.003446,1890.108198,1923.332785,1920.445001,1909.373981,1954.382000,1951.335184,1960.869450
1,22.360680,0.000000,14.142136,41.231056,31.622777,41.243181,50.009999,72.117959,84.858706,67.089492,...,1872.113512,1894.230451,1870.857825,1875.155727,1908.011792,1905.205763,1894.467999,1938.790602,1935.822564,1945.381454
2,36.055513,14.142136,0.000000,50.000000,28.284271,30.016662,50.009999,76.164296,70.717749,53.860932,...,1859.518486,1881.624033,1858.523339,1862.903379,1895.602543,1892.830949,1882.235108,1926.268154,1923.332785,1932.901705
3,28.284271,41.231056,50.000000,0.000000,36.055513,58.318093,31.638584,36.069378,111.807871,86.029065,...,1875.449013,1897.632472,1872.594190,1876.408538,1910.185593,1907.173039,1895.602543,1941.651102,1938.481106,1947.975616
4,41.231056,31.622777,28.284271,36.055513,0.000000,22.383029,22.383029,51.000000,76.164296,50.009999,...,1846.783420,1868.932583,1844.805952,1848.893994,1882.155413,1879.257566,1868.156578,1913.245672,1910.185593,1919.715864


In [16]:
features = {}
for i in np.arange(1,6):
    features[i] = pd.read_table('citydata/features_'+str(i*10)+'.txt',sep=' ',header=None)

In [17]:
non_rate = {}
for i in np.arange(1,6):
    non_rate[i] = pd.read_table('citydata/non_rate_'+str(i*10)+'.txt',sep=' ',header=None)

In [18]:
relation = {}
for i in np.arange(1,6):
    relation[i] = pd.read_table('citydata/relation_'+str(i*10)+'.txt',sep=' ',header=None)

In [21]:
features[1].head()

,0,1,2,3,4,5,6,7,8,9
0,45.0,317.0,292.0,6.0,381.0,42.0,156.0,377.0,41.0,200.0
1,9.0,46.0,338.0,317.0,330.0,18.0,467.0,174.0,280.0,184.0
2,311.0,212.0,227.0,174.0,25.0,219.0,255.0,255.0,321.0,173.0
3,343.0,164.0,202.0,321.0,42.0,131.0,328.0,474.0,79.0,246.0
4,245.0,423.0,419.0,492.0,76.0,270.0,39.0,106.0,146.0,478.0


In [25]:
features[1].shape

(9448, 10)

In [33]:
relation[1].head()

,0
0,0.650466
1,0.646503
2,0.610530
3,0.602485
4,0.595724


In [27]:
non_rate[1].shape

(9448, 1)

In [28]:
features[2].shape

(9998, 20)

In [34]:
ls citydata/

10feature_counts100000test_counts1000point_counts/       non_rate_10.txt
10feature_counts1000test_counts1000point_counts/         non_rate_20.txt
20feature_counts100000test_counts1000point_counts/       non_rate_30.txt
30feature_counts100000test_counts1000point_counts/       non_rate_40.txt
distance.txt                                             non_rate_50.txt
__DOA__10feature_counts1000test_counts1000point_counts/  relation_10.txt
features_10.txt                                          relation_20.txt
features_20.txt                                          relation_30.txt
features_30.txt                                          relation_40.txt
features_40.txt                                          relation_50.txt
features_50.txt


In [35]:
ls citydata/10feature_counts100000test_counts1000point_counts/

first_index.txt  nanrate_specific.txt  pcc.txt                second_index.txt
label.txt        nan_rate.txt          relation_specific.txt


In [36]:
np.loadtxt('citydata/10feature_counts100000test_counts1000point_counts/first_index.txt')

array([[  84., 1092., 1104., ...,  117., 1021.,  857.],
       [ 748., 1026.,  258., ...,  976.,  630., 1091.],
       [1210.,  877.,  526., ..., 1439.,  430., 1420.],
       ...,
       [ 657.,  612.,  356., ...,  670., 1192.,  676.],
       [1245.,  862., 1004., ..., 1292.,  866.,  932.],
       [ 190., 1484.,  704., ...,  154.,  942.,  100.]])

In [37]:
def get_detailed_info(path):
    firstind = np.loadtxt(path+'first_index.txt')
    secondind = np.loadtxt(path+'second_index.txt')
    label = np.loadtxt(path+'label.txt')
    nanrate_specific = np.loadtxt(path+'nanrate_specific.txt')
    nan_rate = np.loadtxt(path+'nan_rate.txt')
    pcc = np.loadtxt(path+'pcc.txt')
    relation_specific = np.loadtxt(path+'relation_specific.txt')
    return firstind, secondind, label, nanrate_specific, nan_rate, pcc, relation_specific

In [38]:
firstind, secondind, label, nanrate_specific, nan_rate, pcc, relation_specific = \
    get_detailed_info('citydata/10feature_counts100000test_counts1000point_counts/')

In [39]:
firstind.shape

(100000, 1000)

In [43]:
pcc.shape

(100000,)